In [4]:
import pandas as pd
import random

In [5]:
# Загружаем файл с разметкой в виде Dataframe
data = pd.read_csv('markup.csv', sep='\t')

# Проверяем
data.head()

,DocName,Tokens,KE_0,KE_1,KE_2,KE_3,KE_4,Blocks_0,Blocks_1,Blocks_2,Blocks_3
0,Протокол ВОСА № 2 от 19_12_2013.html.txt,о,O,O,O,O,O,O,O,O,O
1,Протокол ВОСА № 2 от 19_12_2013.html.txt,',O,O,O,O,O,O,O,O,O
2,Протокол ВОСА № 2 от 19_12_2013.html.txt,«,O,O,O,O,O,O,O,O,O
3,Протокол ВОСА № 2 от 19_12_2013.html.txt,•,O,O,O,O,O,O,O,O,O
4,Протокол ВОСА № 2 от 19_12_2013.html.txt,ДВЭУК,O,O,O,O,O,O,O,O,O


**Подготовительные работы**

In [6]:
# Собираем названия колонок
columns = list(data.columns.values)

# В ручную вводим номер колонки, с которой в файле начинается разметка (может меняться в будущих файлах)
FISRT_OBJECT_COLUMN = 2

# Сохраняем названия рабочих колонок 
object_columns = columns[FISRT_OBJECT_COLUMN:]

# Выводим имена колонок и списки названий объектов в каждой колонке, чтобы глазами посмотреть, что там вообще есть
for name in object_columns:
    print('Column: ' + name)
    print(data[name].unique())
    print()

Column: KE_0
['O' 'B-Орган_ОСА' 'I-Орган_ОСА' 'B-НЮЛ' 'I-НЮЛ' 'B-Орган_Отсутствует'
 'I-Орган_Отсутствует' 'B-Дата' 'I-Дата' 'B-Время' 'I-Время' 'B-ФизЛицо'
 'I-ФизЛицо' 'B-Орган_ЕИО' 'I-Орган_ЕИО' 'B-Орган_СД' 'I-Орган_СД'
 'B-ГосОрган' 'I-ГосОрган' 'B-Орган_ОСУ' 'B-Орган_КИО' 'B-Сумма' 'I-Сумма'
 'I-Орган_ОСУ']

Column: KE_1
['O' 'B-Орган-Форма' 'B-НЮЛ-ОПФ' 'I-НЮЛ-ОПФ' 'B-НЮЛ-Наименование'
 'I-НЮЛ-Наименование' 'B-Сумма-валюта' 'B-Сумма-оператор' 'I-Сумма-валюта'
 'B-Сумма-сумма_прописью' 'I-Сумма-сумма_прописью' 'I-Сумма-оператор']

Column: KE_2
['O' 'B-Должность' 'I-Должность']

Column: KE_3
['O' 'B-Цифры_Номер' 'I-Цифры_Номер' 'B-Адрес' 'I-Адрес' 'B-Цифры_Кол-во'
 'I-Цифры_Кол-во' 'B-Цифры_Процент' 'I-Цифры_Процент'
 'B-Цифры_Номер порядковый' 'I-Цифры_Номер порядковый'
 'B-Цифры_Иная цифра' 'B-Цифры_Длительность' 'I-Цифры_Длительность'
 'I-Цифры_Иная цифра']

Column: KE_4
['O' 'B-Цифры-БазаОпред' 'B-Адрес-СубъектРФ' 'I-Адрес-СубъектРФ'
 'B-Адрес-НасПункт' 'I-Адрес-НасПункт' 'B-Ад

In [7]:
# Делаем функцию для парсинга имени объекта, разделяя реальное имя сущности и метку границ (нужно будет для формирования списка всех объектов)

# Возвращаем пару - в первом элементе: метка границ или 'O', во втором: чистое имя сущности
def parse_name(obj_name):
    if obj_name == 'O':
        return ('O', 'O')
    elif obj_name[0:2] == 'B-':
        return ('B', obj_name[2:])
    elif obj_name[0:2] == 'I-':
        return ('I', obj_name[2:])
    # В случае названий без меток просто возвращаем 'O' и само название
    else:
        return ('O', obj_name)

# Проверяем
parse_name('I-test') == ('I', 'test')

True

In [8]:
# Собираем все сущности в отдельные объекты (словари) со всеми необходимыми атрибутами, чтобы потом записать их в DataFrame
# И здесь же собираем номера начальных и конечных токенов каждого документа (чтобы потом использовать границы документов для распечатки фрагментов)

# Список для всех объектов
object_list = []
# Словарь для границ документов
doc_coords_dict = {}
# Начальное значение для предыдущего объекта
prev_name = 'O'
# Начальне значение для предыдущего документа
prev_doc = 'O'

# !! Этот цикл на предложенном файле (113954 строк) отрабатывал около 1 мин. (в Google Colab) 

# Итерируем сначала по всем рабочим колонкам, затем по всем строкам
for name in object_columns:
    for index, row in data.iterrows():
        # Берем имя сущности на текущей строке
        cur_name = parse_name(row[name])[1]
        # Берем метку границ на текущей строке
        cur_border = parse_name(row[name])[0]
        # Берем название документа на текущей строке
        cur_doc = row['DocName']
        # Берем текст текущего токена (для отладки)
        cur_text = row['Tokens']
        # Отладка
        #print(cur_name, cur_border, index, cur_text)
    
        # Создаем объект по метке начала или по сложному условию на начало нового блока
        if (cur_border == 'B') | ((cur_border == 'O') & (cur_name != 'O') & (cur_name != prev_name)):
            # Создаем новый (текущий) объект
            cur_object = {}
            # Записываем координату начала объекта через текущий индекс (номер строки)
            cur_object['begin'] = index
            # Записываем координату конца, она сначала равна началу, потом будем ее двигать
            cur_object['end'] = index
            # Записываем имя сущности
            cur_object['object_type'] = cur_name
            # Записываем название документа через set, чтобы если вдруг объект переходит через границу документов, 
            # то мы поймали бы все названия документов и увидели бы, что есть такая проблема 
            cur_object['doc_name_set'] = set()
            cur_object['doc_name_set'].add(cur_doc)
            # Ставим метку, является ли объект блоком или нет (от этого будут зависеть дальнейшие преобразования)
            cur_object['is_block'] = (cur_border == 'O')
            # Добавляем текущий объект в общий список
            object_list.append(cur_object)

        # Определяем ситуацию продолжения начатого объекта - либо метка продолжения, либо сложное условие на продолжение блока
        elif (cur_border == 'I') | ((cur_name == prev_name) & (cur_name != 'O')):
            # Сдвигаем координату конца на текущий индекс
            cur_object['end'] = index
            # Продолжаем добавлять название документа на случай, если оно вдруг изменится (тогда поймаем все)
            cur_object['doc_name_set'].add(cur_doc)
        
        # Проверяем, что имя текущего документа отличается от предыдущего  
        # Если отличается, то создаем запись для данного документа с началом на текущем индексе и концом на нуле (концы вычислим позже)
        # Мы пройдем по каждой границе документов по многу раз (по количеству колонок), 
        # но так как это словарь, запишем начальную координату только один раз для каждого документа
        if cur_doc != prev_doc:
            doc_coords_dict[cur_doc] = (index, 0)

        # Переключаем предыдущее имя объекта на текущее
        prev_name = cur_name
        # Переключаем предыдущее имя документа на текущее
        prev_doc = cur_doc
        
        # Отладка
        #if index > 5000:
          #break

# Сохраняем список в Dataframe
object_df = pd.DataFrame(object_list)
# Проверяем
object_df.head()

,begin,end,object_type,doc_name_set,is_block
0,8,11,Орган_ОСА,{Протокол ВОСА № 2 от 19_12_2013.html.txt},False
1,12,20,НЮЛ,{Протокол ВОСА № 2 от 19_12_2013.html.txt},False
2,22,25,НЮЛ,{Протокол ВОСА № 2 от 19_12_2013.html.txt},False
3,31,39,НЮЛ,{Протокол ВОСА № 2 от 19_12_2013.html.txt},False
4,70,70,Орган_ОСА,{Протокол ВОСА № 2 от 19_12_2013.html.txt},False


In [9]:
# Проверяем, что у всех объектов прописался только один документ 
# (для этого мы записывали имена документов в set)
# Здесь же мы увидим случаи, где одноименные блоки стоят друг за другом (в самом конце одного документа и в самом начале следующего)
# Если такие случаи здесь вылезут, то нужно будет добавить механизм разрезания блоков по границам документов в цикле выше 

# Записываем количество документов для данного объекта в отдельную колонку
object_df['docs_number'] = object_df['doc_name_set'].apply(lambda x: len(x))

# Вычисляем, сколько объектов накрывают более одного документа
# Это число уникальных значений количества документов минус 1
num_err = len(object_df['docs_number'].unique()) - 1

# Выводим сообщение с этой информацией
print('Number of objects covering more than 1 document: ', num_err)
print()

# Если таких ошибок нет, то записываем название документа в новую колонку (без set) и удаляем ненужые колонки 
if num_err == 0:
    object_df['doc_name'] = object_df['doc_name_set'].apply(lambda x: next(iter(x)))
    object_df.drop(['docs_number', 'doc_name_set'], axis=1, inplace=True)

# Проверяем 
object_df.head()

Number of objects covering more than 1 document:  0



,begin,end,object_type,is_block,doc_name
0,8,11,Орган_ОСА,False,Протокол ВОСА № 2 от 19_12_2013.html.txt
1,12,20,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt
2,22,25,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt
3,31,39,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt
4,70,70,Орган_ОСА,False,Протокол ВОСА № 2 от 19_12_2013.html.txt


In [10]:
# Делаем функцию для распечатки текста по координатам начала и конца 
# Печатаем через пробелы, это не всегда удачно, например если в токене пунктуатор, но зато просто и достаточно наглядно

def print_by_coord(begin, end, is_block):
    res = ''

    # Если координаты неправильные, возвращаем специальную метку 
    if begin > end:
        return '<Error in coordinates>'
    
    # Если координаты (-1, -1), значит нужно распечатать абсолютное начало документа 
    # Такие координаты будут передаваться в случае, если нужно распечатать левый контекст объекта, который стоит в абсолютном начале документа
    if (begin, end) == (-1, -1):
        return '<Document start>'

    # Если координаты (-2, -2), значит нужно распечатать абсолютный конец документа
    # Такие координаты будут передаваться в случае, если нужно распечатать правый контекст объекта, который стоит в абсолютном конце документа
    if (begin, end) == (-2, -2):
        return '<Document end>'

    # Собираем текст токенов в одну строку
    for i in range(begin, end + 1):
        res += str(data['Tokens'].iloc[i]) + ' '
    # Отрезаем последний пробел
    res = res[:-1]

    # В случае обычных сущностей возвращаем собранный текст
    if not is_block:
        return res
    # В случае блоков возвращаем специальную метку и начало текста (первые 100 символов и многоточие)  
    else:
        return '<Block> ' + res[:100] + ' ...'

# Проверяем
print(print_by_coord(5, 5, False) == 'ПРОТОКОЛ')
print(print_by_coord(6012, 6300, True) == 
      '<Block> ПРОТОКОЛ № 1 годового Общего собрания акционеров Публичного акционерного общества Банк « Возрождение ...')
print(print_by_coord(-1, -1, False) == '<Document start>')

True
True
True


In [11]:
# Добавляем текст объектов по их координатам
object_df['text'] = object_df.apply(lambda x: print_by_coord(x['begin'], x['end'], x['is_block']), axis=1)

# Проверяем 
object_df.head()

,begin,end,object_type,is_block,doc_name,text
0,8,11,Орган_ОСА,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,внеочередного Общего собрания акционеров
1,12,20,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,Открытого акционерного общества « Дальневосточ...
2,22,25,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,ОАО « ДВЭУК »
3,31,39,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,Открытое акционерное общество « Дальневосточна...
4,70,70,Орган_ОСА,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,акционеров


In [12]:
# Довычисляем конечные координаты в словаре документов

# Берем индекс последнего токена всего корпуса
data_end_coord = len(data) - 1

# Берем ключи и значения словаря
keys = list(doc_coords_dict)
values = list(doc_coords_dict.values())

# Перезаписываем словарь заново, добавляя во второй элемент пары конечную координату данного документа (это начальная следующего минус 1)
for i in range(len(keys)):
    # Для всех кроме последнего документа
    if i < len(keys)-1:
        doc_coords_dict[keys[i]] = (values[i][0], values[i+1][0]-1)
    # Для последнего документа
    else:
        doc_coords_dict[keys[i]] = (values[i][0], data_end_coord)

# Проверяем
doc_coords_dict

{'Протокол ВОСА № 2 от 19_12_2013.html.txt': (0, 2943),
 'protocol_gosa_18.01.2019.html.txt': (2944, 6011),
 'protocol_gosa_2029.06.2018.html.txt': (6012, 14693),
 'protocol_31.05.2019.html.txt': (14694, 22587),
 'vbank_protocol25_vosa_070318.html.txt': (22588, 23557),
 'Протокол ВОСА № 2 от 26_12_2017.html.txt': (23558, 25630),
 'protocol_meeting_07032018.html.txt': (25631, 27999),
 'vbank_protocol_vosa_070318.html.txt': (28000, 28975),
 'Протокол ВОСА № 1 от 18_10_2013.html.txt': (28976, 33657),
 'vosa_29092017.html.txt': (33658, 38510),
 'Protocol BOCA_18_06_2018.html.txt': (38511, 43789),
 'protokol_gosa_30 06 2011.html.txt': (43790, 46874),
 'BOCA_protocol_n1_02_04_2012.html.txt': (46875, 49732),
 'Протокол ГОСА_2016.html.txt': (49733, 55269),
 'gosa_27_06_2013.html.txt': (55270, 63417),
 'protokol_vosa_3_ot27_12_2011.html.txt': (63418, 65073),
 'vbank_protokol_2_vosa_2014.html.txt': (65074, 67774),
 'vipiska_8.html.txt': (67775, 68848),
 'itogi_golosovaniya.html.txt': (68849, 700

In [41]:
# Делаем функцию для распечатки сниппета данной сущности (выделенного фрагмента текста с левым и правым контекстом) по ее индексу в object_df
# Передаем индекс объекта и ширину контекста

def print_snippet(ind, context_wide):
    # Проверяем, является ли объект блоком
    is_block = (object_df.iloc[ind]['is_block'] == True)

    # Определяем название документа, чтобы определить начальную и конечную координату
    doc_name = object_df.iloc[ind]['doc_name']
    # Берем начальную и конечную координаты для данного документа, чтобы ограничить ими левый и правый контекст
    min_ind = doc_coords_dict[doc_name][0]
    max_ind = doc_coords_dict[doc_name][1]

    # Берем координаты по индексу
    begin = object_df.iloc[ind]['begin']
    end = object_df.iloc[ind]['end']

    # Вычисляем координаты левого контекста
    # Проверяем, что не выходим за минимальный индекс данного документа влево
    begin_left = max(min_ind, begin - context_wide)
    end_left = max(min_ind, begin - 1)

    # Вычисляем координаты правого контекста
    # Проверяем, что не выходим за максимальный индекс данного документа вправо
    begin_right = min(max_ind, end + 1)
    end_right = min(max_ind, end + context_wide)

    # Отдельно обрабатываем случаи, когда объект в абсолютном начале или абсолютном конце документа
    # В таких случаях будем распечатывать метки начала и конца
    # Удобнее было бы там иметь специальные токены начала и конца документа, как обычно и бывает, но их там нет
    if begin == min_ind:
        begin_left, end_left = -1, -1
    if end == max_ind:
        begin_left, end_left = -2, -2

    # Формируем части сниппета
    left = print_by_coord(begin_left, end_left, False)
    right = print_by_coord(begin_right, end_right, False)
    middle = print_by_coord(begin, end, is_block)

    # Распечатываем общие данные про данный объект (название, документ, индекс) 
    print('Object: ' + str(object_df.iloc[ind]['object_type']) + ' Document: ' + str(object_df.iloc[ind]['doc_name']) + ' Index: ' + str(ind))
    # Распечатываем текст сущности вместе с контекстом с выделением жирным текста сущности
    # Дополнительно печатаем квардратные скобки вокруг сущности (чтобы при дальнейшем копировании сохранялось выделение)
    print(left + ' ' + '\033[1m' + '[' + middle + ']' + '\033[0m' + ' ' + right)
    # Пустая строка для удобства чтения
    print()

# Делаем функцию для распечатки сниппетов по списку индексов
def print_snippets(list_of_indexes, context_wide):
    for i in list_of_indexes:
        print_snippet(i, context_wide)

# Проверяем
print_snippet(1, 5)
print_snippets([1], 5)
# Это объект из начала не первого документа, здесь должен обрезаться левый контекст по границе документа
print_snippet(226, 10)

Object: НЮЛ Document: Протокол ВОСА № 2 от 19_12_2013.html.txt Index: 1
2 внеочередного Общего собрания акционеров [Открытого акционерного общества « Дальневосточная энергетическая управляющая компания »] ( ОАО « ДВЭУК »

Object: НЮЛ Document: Протокол ВОСА № 2 от 19_12_2013.html.txt Index: 1
2 внеочередного Общего собрания акционеров [Открытого акционерного общества « Дальневосточная энергетическая управляющая компания »] ( ОАО « ДВЭУК »

Object: Орган_ОСА Document: protocol_gosa_18.01.2019.html.txt Index: 226
ПРОТОКОЛ № 1 [внеочередного Общего собрания акционеров] Публичного акционерного общества Банк « Возрождение » , проводимого в



In [14]:
# Вычисляем суммарное количество инстансов каждой сущности

# Группируем по типу сущности и выводим количество
object_count = object_df['object_type'].value_counts()
# Записываем в словарь (понадобится потом для вывода информации о данной сущности)
object_count_dict = object_count.to_dict()

# Распечатываем
object_count

Цифры-цифры_номером                                           5064
Цифры_Номер порядковый                                        2462
Цифры_Кол-во                                                  1852
НЮЛ                                                           1798
НЮЛ-Наименование                                              1586
НЮЛ-ОПФ                                                       1566
Цифры-единица_измерения                                       1489
ФизЛицо                                                       1407
Орган_ОСА                                                     1315
Цифры-БазаОпред                                               1146
Должность                                                      944
Дата                                                           936
Цифры-цифры_прописью                                           861
Орган_СД                                                       814
Орган-Форма                                                   

In [15]:
# Для удобства распечатки больших таблиц (ниже) добавляем номера документов, чтобы выводить их в колонки вместо названий

# Берем список имен всех документов
doc_names = (object_df['doc_name'].unique())
# Делаем словарь "имя документа: номер"
docs_dict = {}
for i in range(len(doc_names)):
    docs_dict[doc_names[i]] = i

# Добавляем колонку с номером документа
object_df['doc_index'] = object_df['doc_name'].apply(lambda x: docs_dict[x])

# И так же для удобства добавлям номера сущностей вместо названий
# Берем список имен сущностей
obj_types = object_df['object_type'].unique()
# Делаем словарь "имя сущности: номер"
obj_dict = {}
for i in range(len(obj_types)):
    obj_dict[obj_types[i]] = i

# Добавляем колонку с номером сущности
object_df['obj_index'] = object_df['object_type'].apply(lambda x: obj_dict[x])

# Проверяем
object_df.head()

,begin,end,object_type,is_block,doc_name,text,doc_index,obj_index
0,8,11,Орган_ОСА,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,внеочередного Общего собрания акционеров,0,0
1,12,20,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,Открытого акционерного общества « Дальневосточ...,0,1
2,22,25,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,ОАО « ДВЭУК »,0,1
3,31,39,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,Открытое акционерное общество « Дальневосточна...,0,1
4,70,70,Орган_ОСА,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,акционеров,0,0


In [16]:
# Добавляем длину объекта в токенах
object_df['length'] = object_df.apply(lambda x: x['end'] - x['begin'] + 1, axis=1)

# Проверяем
object_df.head()

,begin,end,object_type,is_block,doc_name,text,doc_index,obj_index,length
0,8,11,Орган_ОСА,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,внеочередного Общего собрания акционеров,0,0,4
1,12,20,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,Открытого акционерного общества « Дальневосточ...,0,1,9
2,22,25,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,ОАО « ДВЭУК »,0,1,4
3,31,39,НЮЛ,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,Открытое акционерное общество « Дальневосточна...,0,1,9
4,70,70,Орган_ОСА,False,Протокол ВОСА № 2 от 19_12_2013.html.txt,акционеров,0,0,1


**Задание 1: аналитика и визуализация**


**Общая таблица по объектам с их текстом**

Самый просто способ визуализации - экспортировать в csv таблицу object_df. В ней получилось 29378 строк (около 4 мб). Далее, например, в Open Office ее можно открыть и с помощью автофильтров (по названиям сущностей, по документам, по длине и тп.) смотреть на тексты разных объектов.

In [17]:
# Экспортируем object_df в csv 
object_df.to_csv('all_objects.csv')

**Таблицы по количеству объектов в документе**

Теперь сделаем еще две обзорные таблицы с информацией о количестве объектов каждого типа в каждом документе.



In [18]:
# Таблица по объектам 
# (в строках объекты, в колонках документы, в клетках количество объектов в документе)

# Вариант для экспорта в csv (в колонках названия документов)
# Группируем по типу объекта и по названию документа и выносим названия документов в колонки с количеством в клетках
stat_by_object_export = object_df.groupby(['object_type', 'doc_name']).size().unstack(fill_value=0)
# Добавляем строку Total с суммой значений по всем строкам для каждой колонки
stat_by_object_export.loc['Total'] = stat_by_object_export.sum(numeric_only=True)
# Экспортируем в csv
stat_by_object_export.to_csv('stat_by_object.csv')

# Вариант для распечатки в блокноте (в колонках номера вместо названий документов, чтобы лучше помещалось в экран)
# Точно так же группируем и выносим колонки, но вместо имен документов берем их индексы
stat_by_object = object_df.groupby(['object_type', 'doc_index']).size().unstack(fill_value=0)
# Добавляем строку Total с суммой значений по всем строкам
stat_by_object.loc['Total'] = stat_by_object.sum(numeric_only=True)

# Распечатываем
stat_by_object

doc_index,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
object_type,,,,,,,,,,,,,,,,,,,,,
10_Блок_Подписи,2,1,1,1,1,2,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
11_Блок_ЗаверениеКопии,0,0,1,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1_Блок_Заголовок,1,1,2,2,1,1,1,1,1,3,...,1,1,2,1,1,1,1,2,1,1
2_Блок_Участники,1,1,2,1,1,1,1,1,1,1,...,1,2,3,2,2,1,1,2,1,1
3_1_ПовесткаДня,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,2,1,1,1
3_1_ПовесткаДня->Номер,5,3,7,10,4,1,2,2,3,5,...,12,3,10,13,2,2,7,11,1,11
3_1_ПовесткаДня->Текст,5,3,7,10,4,1,2,2,3,5,...,12,3,10,13,2,2,7,11,1,11
3_Блок_Повестка,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4_1_ОбсуждениеПовестки,1,1,2,1,1,2,1,1,2,1,...,1,1,1,1,1,1,1,1,1,1


In [19]:
# Таблица по документам
# (в строках документы, в колонках сущности, в клетках количество инстансов данной сущности в документе)

# Вариант для экспорта в csv (в колонках названия сущностей)
# Группируем по названию документа и по типу объекта и выносим типы объектов в колонки с количеством в клетках
stat_by_doc_export = object_df.groupby(['doc_name', 'object_type']).size().unstack(fill_value=0)
# Добавляем строку Total с суммой значений по всем строкам для каждой колонки
stat_by_doc_export.loc['Total'] = stat_by_doc_export.sum(numeric_only=True)
# Экспортируем в csv
stat_by_doc_export.to_csv('stat_by_doc.csv')

# Вариант для распечатки в блокноте (в колонках номера вместо названий сущностей, чтобы лучше помещалось в экран)
# Точно так же группируем и выносим колонки, но вместо имен объектов берем их индексы
stat_by_doc = object_df.groupby(['doc_name', 'obj_index']).size().unstack(fill_value=0)
# Добавляем строку Total с суммой значений по всем строкам для каждой колонки
stat_by_doc.loc['Total'] = stat_by_doc.sum(numeric_only=True)

# Распечатываем
stat_by_doc

obj_index,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
doc_name,,,,,,,,,,,,,,,,,,,,,
180517_rus_minutes_No.37_ВОСА_16.05.2018.html.txt,80,55,32,31,5,93,9,26,0,0,...,1,1,11,11,11,11,11,11,11,11
4. Выписка СД_26.04.2019 Распределение прибыли.html.txt,6,29,0,8,1,17,1,12,0,0,...,1,1,1,1,0,0,0,0,1,1
BOCA_protocol_n1_02_04_2012.html.txt,39,60,4,20,5,14,1,9,2,0,...,1,1,4,4,4,4,4,4,4,4
GOSA15_06_2012.html.txt,36,49,13,32,5,57,10,36,2,0,...,1,2,10,10,10,1,10,3,9,10
GOSA17_06_2014.html.txt,87,58,15,44,5,68,13,46,2,2,...,1,1,11,11,12,11,11,5,11,12
Protocol BOCA_18_06_2018.html.txt,75,74,21,56,5,63,18,35,8,0,...,2,1,10,10,10,10,10,7,10,10
Protocol ВОСА_26_12_2018 № 2.html.txt,25,51,8,15,0,8,0,10,4,0,...,1,1,2,2,2,2,0,1,2,2
cf070284588b587d50022505f8c00285.html.txt,43,87,42,43,7,74,19,40,0,1,...,1,1,13,13,14,13,13,0,13,13
gosa_27_06_2013.html.txt,84,62,15,34,5,65,19,63,1,0,...,1,1,12,12,12,14,12,6,12,11


Используя эти таблицы, можно даже глазами увидеть разные аномалии - редкие объекты, документы, в которых вхождений какого-то объекта сильно больше, чем в других и т.п. Удобнее всего это делать через csv в Open Office, но можно прямо в блокноте.

**Визуализация конкретной сущности по сниппетам**

Допустим, мы нашли в этих таблицах что-то интересное. Например, объект "Цифры_Длительность" в целом встречается редко, но в одном документе - много раз. Хотим посмотреть на него подробнее. Для этого сделаем удобный вывод нескольких сниппетов по сущности (названию или индексу) с возможностью фильтра по документам.


In [20]:
# Делаем функцию для распечатки информации о данной сущности и примеров с ней

def show_object_examples(OBJECT_TO_SHOW, NUMBER_OF_SNIPPETS_TO_SHOW = 3, CONTEXT_WIDE = 5, DOCUMENT_NAMES = []):

    # Обрабатываем входной параметр, который может быть и индексом, и именем сущности
    if type(OBJECT_TO_SHOW) == int:
        obj_name = obj_types[OBJECT_TO_SHOW]
    elif type(OBJECT_TO_SHOW) == str:
        obj_name = OBJECT_TO_SHOW

    # Берем количество инстансов данной сущности во всем корпусе
    obj_count = object_count_dict[obj_name]

    # Фильтруем таблицу со всеми объектами по этому объекту
    # Если список документов не задан, то фильтруем только по имени сущности
    if DOCUMENT_NAMES == []:
        out = object_df[object_df['object_type'] == obj_name]
    # Если список документов задан, то фильтруем еще и по этому списку
    else:
        out = object_df[(object_df['object_type'] == obj_name) & (object_df['doc_name'].isin(DOCUMENT_NAMES))]

    # Берем все отфильтрованные индексы инстансов этого объекта
    obj_indexes = list(out.index.values)

    # Берем произвольное подмножество индексов (количество задается параметром) 
    # Проверяем, что отфильтрованных индексов не меньше параметра, если меньше, то берем все
    if len(obj_indexes) < NUMBER_OF_SNIPPETS_TO_SHOW:
        subset = obj_indexes
    else:
        subset = random.sample(obj_indexes, NUMBER_OF_SNIPPETS_TO_SHOW)

    # Распечатываем заголовок с именем сущности и информацией о количестве
    print('Сущность: ', obj_name)
    print('Показано ' + str(len(subset)) + ' произвольных вхождений из ' + str(obj_count))
    print('Ширина контекста: ' + str(CONTEXT_WIDE))
    print()

    # Распечатываем сниппеты этого объекта по выбранным индексам (ширина контекста задается параметром)
    print_snippets(subset, CONTEXT_WIDE)

# Проверяем
show_object_examples('ФизЛицо')

Сущность:  ФизЛицо
Показано 3 произвольных вхождений из 1407
Ширина контекста: 5

Object: ФизЛицо Document: vbank_protocol25_vosa_070318.html.txt Index: 1691
Антон Александрович ; 9 . [Леонов Роман Александрович] ; 10 . Малинин Сергей

Object: ФизЛицо Document: protocol_meeting_07032018.html.txt Index: 2061
Ивановна 11 443 521 24 [Шамилов Артур Андреевич] 5 953 25 Элик Максим

Object: ФизЛицо Document: vbank_protokol_2_vosa_2014.html.txt Index: 4499
933 329 ; 12 . [Эскиндаров Мухадин Абдурахманович] - 19 933 395 ;



Блок для визуализации конкретной сущности с настраиваемыми параметрами

In [23]:
# Вводим название искомой сущности или ее индекс
OBJECT_TO_SHOW = '1_Блок_Заголовок'
# Вводим количество сниппетов для распечатки
NUMBER_OF_SNIPPETS_TO_SHOW = 5
# Вводим ширину контекста
CONTEXT_WIDE = 2
# Вводим имена документов, по которым фильтруем показ сниппетов (если пусто, то показываем из всех документов)
DOCUMENT_NAMES = []

# Работает в том числе для блоков, но в их случае вместо их текста выводится метка <Block> и виден только контекст, что тоже может быть полезно

show_object_examples(OBJECT_TO_SHOW, NUMBER_OF_SNIPPETS_TO_SHOW, CONTEXT_WIDE, DOCUMENT_NAMES)

Сущность:  1_Блок_Заголовок
Показано 5 произвольных вхождений из 40
Ширина контекста: 2

Object: 1_Блок_Заголовок Document: protocol_gosa_18.01.2019.html.txt Index: 27787
<Document start> [<Block> ПРОТОКОЛ № 1 внеочередного Общего собрания акционеров Публичного акционерного общества Банк « Возрож ...] Категории (

Object: 1_Блок_Заголовок Document: Протокол ГОСА_16_06_2017.html.txt Index: 27927
<Document start> [<Block> « • ДВЭУК ПРОТОКОЛ годового Общего собрания акционеров Акционер н о го об щества « Дальневосточная э ...] Председательствующий на

Object: 1_Блок_Заголовок Document: vosa_29092017.html.txt Index: 27840
г . [<Block> В соответствии с п . 2 ст . 60 Федерального закона « Об акционерных обществах » всем лицам , зарегис ...] Председатель Собрания

Object: 1_Блок_Заголовок Document: protocol_31.05.2019.html.txt Index: 27802
<Document start> [<Block> ПРОТОКОЛ № 1 годового Общего собрания акционеров Публичного акционерного общества Банк « Возрождение ...] Категории (

Object: 1_

**Проблемы разметки, найденные по таблицам с количеством объектов и далее по сниппетам:**


Примеры неконсистентной разметки сущности "Цифры_Длительность":
```
Object: Цифры_Длительность Document: protocol_gosa_2029.06.2018.html.txt
150 % , увеличившись на [48 п . п . по сравнению началом года] . Стоимость риска в 2017

Object: Цифры_Длительность Document: protocol_gosa_2029.06.2018.html.txt
достигла рекордного показателя за последние [7 лет] и составила 13 , 0
```



Примеры неконсистентной разметки сущности "Цифры_Иная цифра":
```
Object: Цифры_Иная цифра Document: protocol_31.05.2019.html.txt
выданных ипотечных кредитов ; - [21] место крупнейших игроков рынка факторинга

Object: Цифры_Иная цифра Document: gosa_27_06_2013.html.txt
страховой суммы убытки , причиненные [третьим лицам] ( любым физическим или юридическим
```


Примеры неконсистентной разметки сущности "Цифры_Номер":
```
Object: Цифры_Номер Document: protocol_gosa_18.01.2019.html.txt
на голосование : Об утверждении [Изменений № 6] , вносимых в Устав Публичного

Object: Цифры_Номер Document: protocol_gosa_18.01.2019.html.txt
. 1 . Утвердить Изменения [№ 6] , вносимые в Устав Публичного
```





**Задание 2**

**2.1 Сущность "Цифры_Номер"**

Посмотрим на распределение длины фрагментов разметки этой сущности.

In [24]:
obj_name = "Цифры_Номер"

# Фильтруем по всем инстансам этой сущности и оставляем только нужные колонки
df_test = object_df[object_df['object_type'] == obj_name][['object_type', 'text', 'length', 'doc_name']].copy()

# Смотрим на первые объекты
df_test.head()

,object_type,text,length,doc_name
12930,Цифры_Номер,ПРОТОКОЛ № 2,3,Протокол ВОСА № 2 от 19_12_2013.html.txt
12933,Цифры_Номер,Протокол № 21,3,Протокол ВОСА № 2 от 19_12_2013.html.txt
12934,Цифры_Номер,Протокол № 26,3,Протокол ВОСА № 2 от 19_12_2013.html.txt
12935,Цифры_Номер,Протокол № 21,3,Протокол ВОСА № 2 от 19_12_2013.html.txt
12941,Цифры_Номер,№ 208 - ФЗ,4,Протокол ВОСА № 2 от 19_12_2013.html.txt


In [25]:
# Выводим статистику по длине этих объектов
df_test['length'].describe()

count    418.000000
mean       5.270335
std        4.931291
min        2.000000
25%        3.000000
50%        3.000000
75%        6.000000
max       34.000000
Name: length, dtype: float64

Видим, что средние значения длины - около 3-5 (что подтверждается примерами из вывода выше в колонке 'text'), но есть и более крупные значения (вплоть до 34).
Подозреваем, что с длинными значениями что-то может быть не так. Проверяем.

In [26]:
# Фильтруем по длине фрагмента с порогом 10
filtered = df_test[df_test['length'] > 10]

# Проверяем
filtered.head()

,object_type,text,length,doc_name
13142,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_18.01.2019.html.txt
13177,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_18.01.2019.html.txt
13238,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_18.01.2019.html.txt
13269,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_2029.06.2018.html.txt
13359,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_2029.06.2018.html.txt


Видим, что в колонке text действительно фрагменты не похожие на те, что были выше (там были фрагменты типа "Протокол № 21", а здесь что-то более сложное)

In [27]:
# Проверяем по сниппетам, что там в тексте

# Берем индексы отфильтрованных инстансов
subset = filtered.index.to_list()

# Выводим несколько первых сниппетов
print_snippets(subset[:3], 5)

Object: Цифры_Номер Document: protocol_gosa_18.01.2019.html.txt Index: 13142
03 . 2018 и требований [ст . 61 Федерального закона « О Центральном Банке России » № 86 - ФЗ от 27 . 06 . 2002 - 23 748 694] . Число голосов , которыми

Object: Цифры_Номер Document: protocol_gosa_18.01.2019.html.txt Index: 13177
03 . 2018 и требований [ст . 61 Федерального закона « О Центральном Банке России » № 86 - ФЗ от 27 . 06 . 2002 - 284 984 328] . Число кумулятивных голосов ,

Object: Цифры_Номер Document: protocol_gosa_18.01.2019.html.txt Index: 13238
03 . 2018 и требований [ст . 61 Федерального закона « О Центральном Банке России » № 86 - ФЗ от 27 . 06 , 2002 - 23 748 694] , Число голосов , которыми



In [28]:
# Для сравнения выводим несколько сниппетов фрагментов с обычной длиной

# Фильтруем по длине меньше 5
filtered_short = df_test[df_test['length'] < 5]

# Берем отфильтрованные индексы
subset_short = filtered_short.index.to_list()

# Выводим несколько первых сниппетов (расширяем контекст до 10, так как даты бьются на несколько токенов)
print_snippets(subset_short[:3], 10)

Object: Цифры_Номер Document: Протокол ВОСА № 2 от 19_12_2013.html.txt Index: 12930
о ' « • ДВЭУК [ПРОТОКОЛ № 2] внеочередного Общего собрания акционеров Открытого акционерного общества « Дальневосточная энергетическая

Object: Цифры_Номер Document: Протокол ВОСА № 2 от 19_12_2013.html.txt Index: 12933
» от 01 . 10 . 2013 г . ( [Протокол № 21] от 02 . 10 . 2013 г . ) .

Object: Цифры_Номер Document: Протокол ВОСА № 2 от 19_12_2013.html.txt Index: 12934
» от 29 . 12 . 2012 г . ( [Протокол № 26] от 29 . 12 . 2012 г . ) .



Видим неконсистентность разметки в этих двух случаях. Дата ("от такого-то числа") в первом случае включается в разметку объекта, а во втором - нет, хотя контексты в этом смысле аналогичны. Например:



```
Object: Цифры_Номер Document: protocol_gosa_18.01.2019.html.txt Index: 13142
03 . 2018 и требований [ст . 61 Федерального закона « О Центральном Банке России » № 86 - ФЗ от 27 . 06 . 2002 - 23 748 694] . Число голосов , которыми

Object: Цифры_Номер Document: Протокол ВОСА № 2 от 19_12_2013.html.txt Index: 12933
» от 01 . 10 . 2013 г . ( [Протокол № 21] от 02 . 10 . 2013 г . ) .
```



Проверяем, в каких случаях у этой сущности максимальные значения длины (там, возможно, еще какие-то другие ошибки).

In [29]:
# Фильтруем с порогом длины 20
filtered_very_long = df_test[df_test['length'] > 20]

# Распечатываем
filtered_very_long

,object_type,text,length,doc_name
13142,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_18.01.2019.html.txt
13177,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_18.01.2019.html.txt
13238,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_18.01.2019.html.txt
13269,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_2029.06.2018.html.txt
13359,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_2029.06.2018.html.txt
13405,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_2029.06.2018.html.txt
13440,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_2029.06.2018.html.txt
13497,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_2029.06.2018.html.txt
13557,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_2029.06.2018.html.txt
13579,Цифры_Номер,ст . 61 Федерального закона « О Центральном Ба...,25,protocol_gosa_2029.06.2018.html.txt


Видим (по колонкам text и length), что есть два типа случаев - с длиной 25 и с длиной 34. Проверим оба случая, посмотрев на сниппеты одного примера на каждую длину. 

In [30]:
# Копируем индексы в ручную из вывода выше
indexes = [14120, 18270]

# Выводим сниппеты
print_snippets(indexes, 10)

Object: Цифры_Номер Document: protocol_meeting_07032018.html.txt Index: 14120
/ 1616 от 06 . 03 . 2018 и требований [ст . 61 Федерального закона « О Центральном Банке России » № 86 - ФЗ от 27 . 06 . 2002 - 163 213 308] . Число кумулятивных голосов , которыми обладают лица , принявшие

Object: Цифры_Номер Document: protokol_2_vosa.html.txt Index: 18270
ВЛ 220 кВ « Городская ( Ленек ) - НПС [№ 12 - НПС № 13 - ПС Олекминск - НПС № 14 » с ПС 220 / 10 кВ при НПС № 12 , ПС 220 / 10 кВ при НПС № 13] ; 2 . 3 . 3 . « Строительство двух



В первом примере видим тот же самый случай, что и выше.

А во втором примере видим, что еще недовыделен текст "НПС" слева от границы размеченного врагмента:


```
Object: Цифры_Номер Document: protokol_2_vosa.html.txt Index: 18270
ВЛ 220 кВ « Городская ( Ленек ) - НПС [№ 12 - НПС № 13 - ПС Олекминск - НПС № 14 » с ПС 220 / 10 кВ при НПС № 12 , ПС 220 / 10 кВ при НПС № 13] ; 2 . 3 . 3 . « Строительство двух


```




**2.2 Сущнось "1_Блок_Заголовок"**

Так как это блок и название у него "Заголовок", то предполагаем, что он должен встречаться в документе не более одного раза (или если документы составные, то по одному разу на каждую часть с заголовком). Проверим это предположение.

In [31]:
obj_name = "1_Блок_Заголовок"

# Сначала посмотрим на количество инстансов этой сущности во всех документах
# Для этого используем уже созданную ранее таблицу stat_by_doc_export
stat_by_doc_export[obj_name]

doc_name
180517_rus_minutes_No.37_ВОСА_16.05.2018.html.txt           2
4. Выписка СД_26.04.2019 Распределение прибыли.html.txt     1
BOCA_protocol_n1_02_04_2012.html.txt                        1
GOSA15_06_2012.html.txt                                     1
GOSA17_06_2014.html.txt                                     1
Protocol BOCA_18_06_2018.html.txt                           1
Protocol ВОСА_26_12_2018 № 2.html.txt                       1
cf070284588b587d50022505f8c00285.html.txt                   1
gosa_27_06_2013.html.txt                                    1
itogi_golosovaniya.html.txt                                 1
itogi_vosa_070318.html.txt                                  1
protocol_31.05.2019.html.txt                                2
protocol_gosa_18.01.2019.html.txt                           1
protocol_gosa_2029.06.2018.html.txt                         2
protocol_meeting_07032018.html.txt                          1
protokol200318.html.txt                                     1

In [43]:
# Фильтруем все документы, в которых таких объектов более 1
filtered = stat_by_doc_export[obj_name][stat_by_doc_export[obj_name] > 1]

# Сохраняем их имена в список и убираем последний элемент (строка Total)
filtered_docs = filtered.index.to_list()[:-1]

# Проверяем
filtered_docs

['180517_rus_minutes_No.37_ВОСА_16.05.2018.html.txt',
 'protocol_31.05.2019.html.txt',
 'protocol_gosa_2029.06.2018.html.txt',
 'vbank_protokol_2_vosa_2014.html.txt',
 'vosa_29092017.html.txt',
 'voting_report_2019.html.txt']

In [34]:
# Посмотрим на эти объекты в этих документах

# Выводим примеры с фильтром по объекту и по списку документов
show_object_examples(obj_name, 20, 5, filtered_docs)

Сущность:  1_Блок_Заголовок
Показано 13 произвольных вхождений из 40
Ширина контекста: 5

Object: 1_Блок_Заголовок Document: protocol_gosa_2029.06.2018.html.txt Index: 27793
<Document start> [<Block> ПРОТОКОЛ № 1 годового Общего собрания акционеров Публичного акционерного общества Банк « Возрождение ...] Почтовые адреса , по которым

Object: 1_Блок_Заголовок Document: protocol_gosa_2029.06.2018.html.txt Index: 27795
, 2 - й Кожевнический [<Block> переулок , дом 12 , строение 2 , ЗАО « РДЦ ПАРИТЕТ » ...] Категории ( типы ) акций

Object: 1_Блок_Заголовок Document: protocol_31.05.2019.html.txt Index: 27802
<Document start> [<Block> ПРОТОКОЛ № 1 годового Общего собрания акционеров Публичного акционерного общества Банк « Возрождение ...] Категории ( типы ) акций

Object: 1_Блок_Заголовок Document: protocol_31.05.2019.html.txt Index: 27804
09 . 2018 г . [<Block> Дата составления протокола Собрания : 03 июня 2019 года . ...] ПОВЕСТКА ДНЯ СОБРАНИЯ : 1

Object: 1_Блок_Заголовок Document: vosa_

Мы видим, что во всех отфильтрованных документах первое вхождение объекта "1_Блок_Заголовок" - это заголовок документа в самом начале (что выглядит логичным), а остальные вхождения - на каких-то других подозрительных фрагментах текста ("Дата составления ...", "Время начала регистрации ..." и прочее). По-видимому, все они являются ошибками разметки. 

**Задание 3**

Чтобы использовать одну сущность для проверки разметки другой, сделаем механизм сравнения пар координат, по которому можно будет проверить, каким образом пересекаются фрагменты разхных сущностей.
Смысл идеи в том, что при правильной разметке не должно быть, например, частичного пересечения блока и не-блока (то есть, не-блоки не должны переходить через границы блоков). В общем случае для любых объектов это необязательно верно, но такие случаи тоже было бы интересно рассмотреть.


In [35]:
# Делаем функцию, которая будет сравнивать две пары координат и определять тип пересечения фрагментов
# Типы пересечения:
# 1) Не пересекаются [ ] { }
# 2) Один фрагмент полностью вложен в другой [ { } ]
# 3) Частичное пересечение фрагментов [ { ] }
# 4) Фрагменты совпадают [{  }]
# 5) Совпадает только одна из границ [  {  }]

def compare_coord(c1, c2):
    # Проверяем, что обе пары координат валидны
    if (c1[0] > c1[1]) | (c2[0] > c2[1]):
        return '<Error in coordinates>'

    # Упорядочиваем фрагменты по начальным координатам (Если на вход пришла пара в обратном порядке, то меняем местами)
    # Это нужно, чтобы ниже перебирать меньше условий
    if c1[0] < c2[0]:
        coord1 = c1
        coord2 = c2
    else:
        coord1 = c2
        coord2 = c1

    begin1 = coord1[0]
    end1 = coord1[1]
    begin2 = coord2[0]
    end2 = coord2[1]
    
    # Условия по типам пересечения (см. выше)
    if (end1 < begin2):
        return 'Не пересекаются'
    elif (begin1 < begin2) & (end2 < end1):
        return 'Полностью вложен'
    elif (begin1 < begin2 <= end1 < end2):
        return 'Частичное пересечение'
    elif (begin1 == begin2) & (end1 == end2):
        return 'Cовпадают'
    elif (begin1 == begin2) | (end1 == end2):
        return 'Совпадает одна из границ'

# Проверяем
print(compare_coord((6,5), (5,5)) == '<Error in coordinates>')
print(compare_coord((1,3), (5,6)) == 'Не пересекаются')
print(compare_coord((1,9), (5,6)) == 'Полностью вложен')
print(compare_coord((1,5), (5,6)) == 'Частичное пересечение')
print(compare_coord((1,5), (1,5)) == 'Cовпадают')
print(compare_coord((1,5), (5,5)) == 'Совпадает одна из границ')

True
True
True
True
True
True


In [44]:
# Возьмем пары координат двух групп сущностей - блоков и не-блоков

# Списки для пар координат
object1_coord_list = []
object2_coord_list = []

# Фильтруем объекты по параметру блок или не-блок
filtered1 = object_df[object_df['is_block'] == True]
filtered2 = object_df[object_df['is_block'] == False]

# Записываем списки пар координат для двух групп
for i in range(len(filtered1)):
    begin = filtered1.iloc[i]['begin']
    end = filtered1.iloc[i]['end']
    object1_coord_list.append((begin, end))

for i in range(len(filtered2)):
    begin = filtered2.iloc[i]['begin']
    end = filtered2.iloc[i]['end']
    object2_coord_list.append((begin, end))

# Проверяем и выводим отладочную информацию о списках
print('Сущности группы 1: ' + str(object1))
print('Количество объектов: ' + str(len(object1_coord_list)))
print('Список координат (начало): ' + str(object1_coord_list[:5]))
print()
print('Сущности группы 2: ' + str(object2))
print('Количество объектов: ' + str(len(object2_coord_list)))
print('Список координат (начало): ' + str(object2_coord_list[:5]))

Сущности группы 1: ['1_Блок_Заголовок', '2_Блок_Участники', '3_Блок_Повестка', '4_Блок_Обсуждение', '10_Блок_Подписи', '8_Блок_ПереченьПриложений', '11_Блок_ЗаверениеКопии', '7_Блок_ДопИнформация', '4_Блок_Обсуждение->РезГолосЕдиножды', '3_1_ПовесткаДня', '4_1_ОбсуждениеПовестки']
Количество объектов: 278
Список координат (начало): [(5, 198), (199, 573), (574, 649), (650, 2896), (2897, 2907)]

Сущности группы 2: ['Орган_ОСА', 'НЮЛ', 'Орган_Отсутствует', 'Дата', 'Время', 'ФизЛицо', 'Орган_ЕИО', 'Орган_СД', 'ГосОрган', 'Орган_ОСУ', 'Орган_КИО', 'Сумма', 'Орган-Форма', 'НЮЛ-ОПФ', 'НЮЛ-Наименование', 'Сумма-валюта', 'Сумма-оператор', 'Сумма-сумма_прописью', 'Должность', 'Цифры_Номер', 'Адрес', 'Цифры_Кол-во', 'Цифры_Процент', 'Цифры_Номер порядковый', 'Цифры_Иная цифра', 'Цифры_Длительность', 'Цифры-БазаОпред', 'Адрес-СубъектРФ', 'Адрес-НасПункт', 'Адрес-Улица', 'Адрес-Дом', 'Цифры-цифры_номером', 'Цифры-цифры_прописью', 'Цифры-единица_измерения', 'Адрес-Индекс', 'Цифры-оператор', '3_1_Пов

In [45]:
# Смотрим на типы пересечения всех объектов из группы 1 со всеми объектами из группы 2 

# Список для результатов
intersect_list = []

# Итерируем по всем элементам обоих списков
# !! Отрабатывает за 17 сек. на всех блоках и не-блоках (в google colab)
for coord1 in object1_coord_list:
    for coord2 in object2_coord_list:
        intersection = {}
        # Берем координаты и записываем тип их пересечения в список результатов
        intersection['coord1'] = coord1
        intersection['coord2'] = coord2
        intersection['intersection_type'] = compare_coord(coord1, coord2)
        intersect_list.append(intersection)

# Записываем в Dataframe
intersect_df = pd.DataFrame(intersect_list)

# Проверяем
intersect_df.head()

,coord1,coord2,intersection_type
0,"(5, 198)","(8, 11)",Полностью вложен
1,"(5, 198)","(12, 20)",Полностью вложен
2,"(5, 198)","(22, 25)",Полностью вложен
3,"(5, 198)","(31, 39)",Полностью вложен
4,"(5, 198)","(70, 70)",Полностью вложен


In [46]:
# Выводим статистику по типам пересечения
intersect_df['intersection_type'].value_counts()

Не пересекаются             8036685
Полностью вложен              52603
Совпадает одна из границ        505
Частичное пересечение             7
Name: intersection_type, dtype: int64

Теперь нужно посмотреть глазами на все пары объектов с частичным пересечением. Для этого выведем сниппеты всех таких пар.

In [39]:
# Записываем пары координат в списки 
coor1_list_intersection3 = intersect_df[intersect_df['intersection_type'] == 'Частичное пересечение']['coord1'].to_list()
coor2_list_intersection3 = intersect_df[intersect_df['intersection_type'] == 'Частичное пересечение']['coord2'].to_list()

# Проверяем (галазами должны быть видны частичные пересечения в парах)
print(coor1_list_intersection3)
print(coor2_list_intersection3)

[(6260, 6311), (6260, 6311), (6312, 6324), (6312, 6324), (63185, 63208), (99334, 99555), (99556, 99655)]
[(6300, 6318), (6308, 6312), (6300, 6318), (6308, 6312), (63204, 63225), (99555, 99558), (99555, 99558)]


In [40]:
# Формируем списки id объектов
object_id_list1 = []
object_id_list2 = []

# Восстанавливаем id объектов по паре координат (предполагаем, что нет разных объектов с одинаковыми координатами, хотя это необязательно верно) 
for coord_pair in coor1_list_intersection3:
    object_id = object_df[(object_df['begin'] == coord_pair[0]) & (object_df['end'] == coord_pair[1])].index.item()
    object_id_list1.append(object_id)

for coord_pair in coor2_list_intersection3:
    object_id = object_df[(object_df['begin'] == coord_pair[0]) & (object_df['end'] == coord_pair[1])].index.item()
    object_id_list2.append(object_id)

# Выводим сниппеты по этим спискам (для удобства выводим попарно, чтобы объекты с пересечениями оказались рядом)
for i in range(len(object_id_list1)):
    print_snippet(object_id_list1[i], 5)
    print_snippet(object_id_list2[i], 5)

Object: 2_Блок_Участники Document: protocol_gosa_2029.06.2018.html.txt Index: 27794
: 05 июня 2018 года [<Block> Почтовые адреса , по которым направлялись заполненные бюллетени для голосования : Российская Федерац ...] переулок , дом 12 ,

Object: Адрес Document: protocol_gosa_2029.06.2018.html.txt Index: 13263
Возрождение » ( ПАО ) [Российская Федерация , 115114 , город Москва , 2 - й Кожевнический переулок , дом 12 , строение 2] , ЗАО « РДЦ ПАРИТЕТ

Object: 2_Блок_Участники Document: protocol_gosa_2029.06.2018.html.txt Index: 27794
: 05 июня 2018 года [<Block> Почтовые адреса , по которым направлялись заполненные бюллетени для голосования : Российская Федерац ...] переулок , дом 12 ,

Object: Адрес-Улица Document: protocol_gosa_2029.06.2018.html.txt Index: 19188
115114 , город Москва , [2 - й Кожевнический переулок] , дом 12 , строение

Object: 1_Блок_Заголовок Document: protocol_gosa_2029.06.2018.html.txt Index: 27795
, 2 - й Кожевнический [<Block> переулок , дом 12 , строение 2 , З

Видим, что в каждой паре действительно один из объектов имеет ошибки в разметке (неверное положение одной из границ). Причем ошибки есть как в блоках, так и в не-блоках.

Ошибки разметки из результатов выше:

```
Object: 2_Блок_Участники Document: protocol_gosa_2029.06.2018.html.txt Index: 27794
: 05 июня 2018 года [<Block> Почтовые адреса , по которым направлялись заполненные бюллетени для голосования : Российская Федерац ...] переулок , дом 12 ,
```


(Неверное положение правой границы блока)

```
Object: 1_Блок_Заголовок Document: protocol_gosa_2029.06.2018.html.txt Index: 27795
, 2 - й Кожевнический [<Block> переулок , дом 12 , строение 2 , ЗАО « РДЦ ПАРИТЕТ » ...] Категории ( типы ) акций
```
(Неверное положение левой границы блока)


```
Object: ФизЛицо Document: gosa_27_06_2013.html.txt Index: 4183
^ ^ ^ ^ / [Гепалов К . В . f , ■ - . ■ ■ ■ Лгг ? et ' w " С - "] Й1 в ; V '
```

(Неверное положение правой границы объекта)



```
Object: 1_Блок_Заголовок Document: 180517_rus_minutes_No.37_ВОСА_16.05.2018.html.txt Index: 27970
www . trcont . ru [<Block> ПРОТОКОЛ № 37 годового Общего собрания акционеров публичного акционерного общества « Центр по перево ...] часов 50 минут по московскому

```
(Неверное положение правой границы блока)



```
Object: 2_Блок_Участники Document: 180517_rus_minutes_No.37_ВОСА_16.05.2018.html.txt Index: 27971
начала подсчета голосов : 11 [<Block> часов 50 минут по московскому времени . Время закрытия годового Общего собрания акционеров : 12 часо ...] Дата составления протокола годового Общего

```
(Неверное положение левой границы блока)


